# Preprocessing

Short idea: load data from `data/raw/`, clean it, transform it, and save the final dataset to `data/processed/`.

In [1]:
import pandas as pd
df_S = pd.read_csv("../data/raw/Superconductivity.csv")
print(df_S.shape)
display(df_S.head(10))

(21263, 82)


,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,...,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,29.0
1,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,...,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606,26.0
2,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,...,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697,19.0
3,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,...,2.264286,2.213364,2.226222,1.368922,1.048834,1,1.100000,0.433013,0.440952,22.0
4,4,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.90607,27.848743,51.968828,...,2.242857,2.213364,2.206963,1.368922,1.096052,1,1.057143,0.433013,0.428809,23.0
5,4,88.944468,57.795044,66.361592,36.098926,1.181795,1.225203,122.90607,20.687458,51.968828,...,2.214286,2.213364,2.181543,1.368922,1.141474,1,1.000000,0.433013,0.410326,23.0
6,4,88.944468,57.682296,66.361592,36.069470,1.181795,1.316857,122.90607,10.765639,51.968828,...,2.142857,2.213364,2.119268,1.368922,1.194453,1,0.857143,0.433013,0.349927,11.0
7,4,76.517718,57.175142,59.310096,35.891368,1.197273,0.943560,122.90607,36.451199,44.289459,...,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697,33.0
8,4,76.517718,56.808817,59.310096,35.773432,1.197273,0.981880,122.90607,34.833160,44.289459,...,2.264286,2.213364,2.226222,1.368922,1.048834,1,1.100000,0.433013,0.440952,36.0
9,4,76.517718,56.442492,59.310096,35.655884,1.197273,1.016495,122.90607,33.215121,44.289459,...,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,31.0


In [15]:
df_SI = pd.read_csv("../data/raw/StabIndex.csv")
print(df_SI.shape)
display(df_SI.head(10))

(10000, 14)


,id,star_luminosity,cosmic_radiation,orbital_velocity,plasma_density,solar_wind_pressure,hull_temperature,magnetic_field_strength,dark_matter_flux,nebula_density,photon_noise_level,engine_thrust,gravity_well_depth,cosmic_stability_index
0,0,2.453146,85.252845,27.585085,3.166404,2.847117,282.836208,17.568126,1.095321,0.001201,29.829353,550.674704,18.384444,10.503134
1,1,1.476078,109.912572,42.697083,5.323041,1.372428,300.924087,12.345116,1.865132,0.011327,31.798827,823.322631,11.489374,4.155593
2,2,2.768071,90.263354,17.792897,0.619088,1.854916,286.596235,26.831654,-1.228111,0.045800,58.841077,507.989352,19.956316,8.415010
3,3,5.575734,142.553658,12.098395,6.065201,5.152423,353.392375,18.349649,1.229622,0.025874,61.383184,398.586939,10.486091,9.334599
4,4,1.367080,232.880829,17.245781,8.439250,2.936329,203.314696,11.787561,3.698667,0.017891,27.263739,488.386018,4.249932,9.801331
5,5,1.367098,79.151975,9.541714,3.813539,2.540310,315.875551,21.162851,-2.893556,0.025201,56.718681,228.472534,19.610657,6.571118
6,6,5.832060,223.663872,46.701294,5.349743,6.574452,273.913043,28.596420,2.320981,0.004663,92.044062,942.458197,4.234108,12.351316
7,7,3.046361,215.632215,44.268087,5.229434,5.059816,303.014352,16.027917,1.077046,0.008111,62.559299,827.454726,11.955921,8.755923
8,8,1.132492,212.450700,18.882127,1.975473,3.790586,362.266016,5.815902,1.644431,0.032290,80.667934,466.040295,14.802900,16.230867
9,9,2.544790,136.754054,16.114452,3.760245,1.579752,257.145988,13.797707,0.958647,0.001216,79.683365,389.366471,10.198141,6.350810


In [16]:
df_Ch = pd.read_csv("../data/raw/ChocolateSales.csv")
print(df_Ch.shape)
display(df_Ch.head(10))

(750, 27)


,id,Id,Web_GRP,TV_GRP,Facebook_GRP,Tone_of_Ad,No_of_Web_Banners,Weather,Avg_Temperature,No_of_Rabbits,...,Import_Regulations,Time_in_Region,Percent_Internet_Access,Percent_Uni_Degrees,Percent_Unemployed,Gender,Coffee_Consumption,Avg_No_of_Cust_Complaints,Avg_Customer_Age,sales
0,327,327,99,56,74,funny,4,rainy,21,94,...,1,92,57,32,7,0,high,235,20,461825.38
1,58,58,100,53,74,emotional,2,rainy,21,100,...,1,54,44,34,9,0,low,246,67,187155.94
2,8,8,101,44,88,emotional,4,cloudy,20,88,...,0,56,50,31,3,1,low,245,77,188302.23
3,866,866,102,53,65,funny,2,rainy,20,115,...,0,98,57,37,4,0,low,224,35,516640.99
4,525,525,101,42,73,funny,5,sunny,14,103,...,1,66,46,33,2,0,low,227,45,250103.04
5,321,321,100,30,74,funny,2,cloudy,29,118,...,1,70,59,31,2,1,high,225,50,275189.21
6,53,53,97,43,95,funny,1,rainy,12,95,...,0,85,48,38,1,0,medium,250,54,418223.20
7,236,236,99,55,87,emotional,1,cloudy,22,113,...,1,67,49,30,8,1,low,245,74,282337.16
8,896,896,101,49,76,funny,4,rainy,22,109,...,1,81,50,35,15,0,low,211,21,379116.73
9,482,482,103,49,74,funny,2,cloudy,18,99,...,0,86,52,32,1,0,low,230,73,412192.73


In [2]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from pathlib import Path

# Paths (this works when you run from the notebooks folder)
raw_dir = Path("../data/raw")
processed_dir = Path("../data/processed")
processed_dir.mkdir(parents=True, exist_ok=True)

# Clean + save all CSVs in raw
for csv_path in raw_dir.glob("*.csv"):
    df = pd.read_csv(csv_path)  # (ya lo tienes cargado)

    # 1) Separar columnas por tipo
    cat_cols = df.select_dtypes(include=["object", "category", "bool"]).columns
    num_cols = df.select_dtypes(include=["number"]).columns

    # 2) Definir transformaciones:
    # - OneHot a categóricas
    # - StandardScaler a numéricas
    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols),
            ("num", StandardScaler(), num_cols)
        ],
        remainder="drop"  # cambia a "passthrough" si quieres mantener otras columnas
    )

    # 3) Aplicar transformaciones
    X = preprocessor.fit_transform(df)

    # 4) Recuperar nombres de columnas para volver a DataFrame
    feature_names = preprocessor.get_feature_names_out()
    df_transformed = pd.DataFrame(X, columns=feature_names, index=df.index)

    df_transformed.head()
    out_path = processed_dir / csv_path.name  # same name
    # out_path = processed_dir / f"{csv_path.stem}_clean.csv"  # <- if you prefer suffix

    df_transformed.to_csv(out_path, index=False)
    print(f"Saved: {out_path} | shape: {df_transformed.shape}")

Saved: ..\data\processed\ChocolateSales.csv | shape: (750, 33)
Saved: ..\data\processed\StabIndex.csv | shape: (10000, 14)
Saved: ..\data\processed\Superconductivity.csv | shape: (21263, 82)
